In [1]:
%pip install -q langchain langchain-openai langchain-community chromadb langchainhub

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q python-dotenv
from dotenv import load_dotenv
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [3]:
#################################################################
# Select an embeddings model
#################################################################
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [4]:

#  pip install chromadb
# https://python.langchain.com/docs/integrations/vectorstores/chroma#passing-a-chroma-client-into-langchain

import chromadb

chroma_client = chromadb.PersistentClient(path="./chromadb")
from langchain_community.vectorstores import Chroma

vectorstore = Chroma(
    # persist_directory="./chromadb",  # by default it is memory only
    collection_name="my_langchain",  # the default chromadb collection is langchain
    client=chroma_client,
    embedding_function=embeddings,  # different from  Chroma.from_documents
    # https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/vectorstores/chroma.py
    # client_settings
)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
# docs = retriever.get_relevant_documents("What is devopsdays?")
# print(docs)

In [5]:

#################################################################
# Get a predefined prompt for RAG
# from the langchain hub
# https://smith.langchain.com/hub
#################################################################

# needs pip install langchainhub
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

Now let's inspect the prompt we got.

In [6]:
from pprint import pprint
pprint(prompt.input_variables)
a=prompt.format_prompt(context="some context",question="some question")
for msg in a.messages:
    print(msg.content)

['context', 'question']
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: some question 
Context: some context 
Answer:


In [7]:
# Configure a Chat LLM
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(temperature=0)

In [8]:

# helper function to join a set of documents
# Retrieved from the vectorstore
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [9]:

#################################################################
# Setup our rag chain
# Using LCEL : Langchain  Expression Language
# https://python.langchain.com/docs/expression_language/
#################################################################
from langchain_core.runnables import RunnablePassthrough

from langchain_core.output_parsers.string import StrOutputParser
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chat
    | StrOutputParser()
)

answer = rag_chain.invoke("What is Devopsdays ?")
print(answer)


Devopsdays is a worldwide series of technical conferences that cover topics of software development, IT infrastructure operations, and the intersection between them. These events are run by volunteers from the local area and often include curated talks and self-organized open space content. Topics discussed at Devopsdays events include automation, testing, security, and organizational culture.
